<a href="https://colab.research.google.com/github/carvalheiracarlos/deep_leaning_notebooks/blob/main/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data()

In [6]:
print(f'Train Sets Shapes.....X_train: {X_train.shape}')
print(f'Train Sets Shapes.....y_train: {y_train.shape}')
print(f'Tests Sets Shapes.....X_test: {X_test.shape}')
print(f'Tests Sets Shapes.....y_test: {y_test.shape}')

# Probably need to merge this X_train and X_test, 
# than partition again in some better proportion(train: 80% | test: 20%) 

Train Sets Shapes.....X_train: (25000,)
Train Sets Shapes.....y_train: (25000,)
Tests Sets Shapes.....X_test: (25000,)
Tests Sets Shapes.....y_test: (25000,)


In [15]:
'''
partition_size: size based on Lenght on X_train
                to divide validation and train sets on proportion of 20% - 80%
'''

partition_size = int(len(X_train) * 0.2)
validation_reviews, validation_labels = X_train[:partition_size], y_train[:partition_size]
train_reviews, train_labels = X_train[partition_size:], y_train[partition_size:]

print(f'Validation Sets: {validation_reviews.shape}, {validation_labels.shape}')
print(f'Train Sets: {train_reviews.shape}, {train_labels.shape}')

Validation Sets: (5000,), (5000,)
Train Sets: (20000,), (20000,)
